In [1]:
import sys
sys.path

['',
 '/usr/lib/python2.7/site-packages/avro-1.7.7-py2.7.egg',
 '/usr/lib64/python27.zip',
 '/usr/lib64/python2.7',
 '/usr/lib64/python2.7/plat-linux2',
 '/usr/lib64/python2.7/lib-tk',
 '/usr/lib64/python2.7/lib-old',
 '/usr/lib64/python2.7/lib-dynload',
 '/home/kranthidr/.local/lib/python2.7/site-packages',
 '/usr/lib64/python2.7/site-packages',
 '/usr/lib64/python2.7/site-packages/gtk-2.0',
 '/usr/lib/python2.7/site-packages',
 '/usr/lib/python2.7/site-packages/IPython/extensions',
 '/home/kranthidr/.ipython']

In [2]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch28Recommendations").getOrCreate()

In [4]:
sc = spark.sparkContext
sc._jsc.sc().uiWebUrl().get()

u'http://gw02.itversity.com:4040'

In [5]:
for x in sc._conf.getAll():
    if 'PROXY' in x[0]:
        print(x[1])
print(spark.conf.get('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES'))
print(spark.conf.get('spark.driver.appUIAddress'))

http://rm01.itversity.com:19288/proxy/application_1533622723243_11704
rm01.itversity.com
http://rm01.itversity.com:19288/proxy/application_1533622723243_11704
http://gw02.itversity.com:4040


In [6]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [9]:
# ratings = spark.read.text("/data/sample_movielens_ratings.txt")\
#   .rdd.toDF()\                           #WHY this Step ??
#   .selectExpr("split(value , '::') as col")\
#   .selectExpr(
#     "cast(col[0] as int) as userId",
#     "cast(col[1] as int) as movieId",
#     "cast(col[2] as float) as rating",
#     "cast(col[3] as long) as timestamp")

In [10]:
ratings1 = spark.read.text("/user/kranthidr/dataSets/spark-guide/data/sample_movielens_ratings.txt")

In [11]:
type(ratings1)

pyspark.sql.dataframe.DataFrame

In [12]:
ratings1.show()

+--------------------+
|               value|
+--------------------+
| 0::2::3::1424380312|
| 0::3::1::1424380312|
| 0::5::2::1424380312|
| 0::9::4::1424380312|
|0::11::1::1424380312|
|0::12::2::1424380312|
|0::15::1::1424380312|
|0::17::1::1424380312|
|0::19::1::1424380312|
|0::21::1::1424380312|
|0::23::1::1424380312|
|0::26::3::1424380312|
|0::27::1::1424380312|
|0::28::1::1424380312|
|0::29::1::1424380312|
|0::30::1::1424380312|
|0::31::1::1424380312|
|0::34::1::1424380312|
|0::37::1::1424380312|
|0::41::2::1424380312|
+--------------------+
only showing top 20 rows



In [13]:
ratings = ratings1.selectExpr("split(value , '::') as col")\
  .selectExpr(
    "cast(col[0] as int) as userId",
    "cast(col[1] as int) as movieId",
    "cast(col[2] as float) as rating",
    "cast(col[3] as long) as timestamp")

In [14]:
ratings.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     0|      2|   3.0|1424380312|
|     0|      3|   1.0|1424380312|
|     0|      5|   2.0|1424380312|
|     0|      9|   4.0|1424380312|
|     0|     11|   1.0|1424380312|
|     0|     12|   2.0|1424380312|
|     0|     15|   1.0|1424380312|
|     0|     17|   1.0|1424380312|
|     0|     19|   1.0|1424380312|
|     0|     21|   1.0|1424380312|
|     0|     23|   1.0|1424380312|
|     0|     26|   3.0|1424380312|
|     0|     27|   1.0|1424380312|
|     0|     28|   1.0|1424380312|
|     0|     29|   1.0|1424380312|
|     0|     30|   1.0|1424380312|
|     0|     31|   1.0|1424380312|
|     0|     34|   1.0|1424380312|
|     0|     37|   1.0|1424380312|
|     0|     41|   2.0|1424380312|
+------+-------+------+----------+
only showing top 20 rows



In [15]:
training, test = ratings.randomSplit([1.8, 0.2])

In [16]:
# Number of Alternating Iteration to solve A = XY equation

als = ALS(rank=10, 
          maxIter=10, 
          regParam=0.01, 
          numUserBlocks=10, 
          numItemBlocks=10, 
          implicitPrefs=False, 
          alpha=1.0, 
          userCol='userId', 
          itemCol='movieId', 
          seed=None, 
          ratingCol='rating', 
          nonnegative=False, 
          checkpointInterval=10, 
          intermediateStorageLevel='MEMORY_AND_DISK', 
          finalStorageLevel='MEMORY_AND_DISK', 
          coldStartStrategy='drop')

# als = ALS().setsetMaxIter(5).setRegParam(0.01).setUserCol("userId").setItemCol("movieId").setRatingCol("rating")

In [17]:
print(als.explainParams())

alpha: alpha for implicit preference (default: 1.0, current: 1.0)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10, current: 10)
coldStartStrategy: strategy for dealing with unknown or new users/items at prediction time. This may be useful in cross-validation or production scenarios, for handling user/item ids the model has not seen in the training data. Supported values: 'nan', 'drop'. (default: nan, current: drop)
finalStorageLevel: StorageLevel for ALS model factors. (default: MEMORY_AND_DISK, current: MEMORY_AND_DISK)
implicitPrefs: whether to use implicit preference (default: False, current: False)
intermediateStorageLevel: StorageLevel for intermediate datasets. Cannot be 'NONE'. (default: MEMORY_AND_DISK, current: MEMORY_AND_DISK)
itemCol: column name for item ids. Ids mus

In [18]:
alsModel = als.fit(training)
predictions = alsModel.transform(test)

In [19]:
# COMMAND ----------
user_recommend = alsModel.recommendForAllUsers(10)

In [20]:
user_recommend.take(2)

[Row(userId=28, recommendations=[Row(movieId=22, rating=5.272603988647461), Row(movieId=92, rating=4.940300941467285), Row(movieId=81, rating=4.791512489318848), Row(movieId=53, rating=4.451572418212891), Row(movieId=2, rating=4.201484680175781), Row(movieId=49, rating=4.0681843757629395), Row(movieId=4, rating=4.064037322998047), Row(movieId=9, rating=4.0637125968933105), Row(movieId=0, rating=3.9807121753692627), Row(movieId=89, rating=3.9679691791534424)]),
 Row(userId=26, recommendations=[Row(movieId=87, rating=6.237438678741455), Row(movieId=18, rating=5.381385803222656), Row(movieId=23, rating=5.07424783706665), Row(movieId=88, rating=5.032304286956787), Row(movieId=22, rating=4.914875507354736), Row(movieId=7, rating=4.892995357513428), Row(movieId=8, rating=4.831716537475586), Row(movieId=24, rating=4.75657844543457), Row(movieId=70, rating=4.655947208404541), Row(movieId=32, rating=4.518425464630127)])]

In [21]:
user_recommend.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [22]:
user_recommend.show(1)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    28|[[22, 5.272604], ...|
+------+--------------------+
only showing top 1 row



In [23]:
user_recommend.selectExpr("userId", "explode(recommendations)").show()    

+------+---------------+
|userId|            col|
+------+---------------+
|    28| [22, 5.272604]|
|    28| [92, 4.940301]|
|    28|[81, 4.7915125]|
|    28|[53, 4.4515724]|
|    28| [2, 4.2014847]|
|    28|[49, 4.0681844]|
|    28| [4, 4.0640373]|
|    28| [9, 4.0637126]|
|    28| [0, 3.9807122]|
|    28|[89, 3.9679692]|
|    26|[87, 6.2374387]|
|    26| [18, 5.381386]|
|    26| [23, 5.074248]|
|    26|[88, 5.0323043]|
|    26|[22, 4.9148755]|
|    26| [7, 4.8929954]|
|    26| [8, 4.8317165]|
|    26|[24, 4.7565784]|
|    26| [70, 4.655947]|
|    26|[32, 4.5184255]|
+------+---------------+
only showing top 20 rows



In [24]:
item_recommend  = alsModel.recommendForAllItems(10)

In [25]:
item_recommend.printSchema()

root
 |-- movieId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- userId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [26]:
item_recommend.first()

Row(movieId=31, recommendations=[Row(userId=23, rating=3.7352795600891113), Row(userId=11, rating=3.721557378768921), Row(userId=12, rating=3.6462903022766113), Row(userId=9, rating=3.2979938983917236), Row(userId=7, rating=2.912750720977783), Row(userId=8, rating=2.795339822769165), Row(userId=14, rating=2.6506643295288086), Row(userId=6, rating=2.6076295375823975), Row(userId=17, rating=2.324929714202881), Row(userId=29, rating=2.1862659454345703)])

In [27]:
item_recommend.selectExpr("movieId", "explode(recommendations)").show()

+-------+---------------+
|movieId|            col|
+-------+---------------+
|     31|[23, 3.7352796]|
|     31|[11, 3.7215574]|
|     31|[12, 3.6462903]|
|     31|  [9, 3.297994]|
|     31| [7, 2.9127507]|
|     31| [8, 2.7953398]|
|     31|[14, 2.6506643]|
|     31| [6, 2.6076295]|
|     31|[17, 2.3249297]|
|     31| [29, 2.186266]|
|     85|[16, 4.7428117]|
|     85|  [8, 4.590405]|
|     85| [14, 4.306075]|
|     85| [7, 4.0199676]|
|     85| [6, 3.5363104]|
|     85|[24, 3.1321023]|
|     85| [17, 3.100744]|
|     85| [1, 2.9121265]|
|     85|[21, 2.8843079]|
|     85| [0, 2.2993433]|
+-------+---------------+
only showing top 20 rows



In [28]:
# COMMAND ----------
from pyspark.ml.evaluation import RegressionEvaluator

In [29]:
evaluator = RegressionEvaluator().setMetricName("rmse").setLabelCol("rating").setPredictionCol("prediction")

In [30]:
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = %f" % rmse)

Root-mean-square error = 1.227566


In [31]:
# COMMAND ----------
from pyspark.mllib.evaluation import RegressionMetrics

In [32]:
predictions.first()

Row(userId=8, movieId=31, rating=3.0, timestamp=1424380312, prediction=2.795339822769165)

In [33]:
predictions.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- prediction: float (nullable = false)



In [34]:
regComparisonDF = predictions.select("rating", "prediction")

In [35]:
regComparisonDF.show(2)

+------+----------+
|rating|prediction|
+------+----------+
|   3.0| 2.7953398|
|   1.0|  2.186266|
+------+----------+
only showing top 2 rows



In [36]:
regComparisonDF.select("rating", "prediction").take(2)

[Row(rating=3.0, prediction=2.795339822769165),
 Row(rating=1.0, prediction=2.1862659454345703)]

In [37]:
from pyspark.sql.functions import struct, create_map, col

In [38]:
regComparisonDF.selectExpr("struct(rating, prediction) as struct", "*").show(2)

+----------------+------+----------+
|          struct|rating|prediction|
+----------------+------+----------+
|[3.0, 2.7953398]|   3.0| 2.7953398|
| [1.0, 2.186266]|   1.0|  2.186266|
+----------------+------+----------+
only showing top 2 rows



In [39]:
regComparisonDF.selectExpr("struct(rating, prediction) as struct").show(2)

+----------------+
|          struct|
+----------------+
|[3.0, 2.7953398]|
| [1.0, 2.186266]|
+----------------+
only showing top 2 rows



In [40]:
regComparison = regComparisonDF.select(create_map(col("rating"), col("prediction")).alias("mapped"))

In [41]:
regComparison.show(3)

+------------------+
|            mapped|
+------------------+
|[3.0 -> 2.7953398]|
| [1.0 -> 2.186266]|
|[1.0 -> 1.1195515]|
+------------------+
only showing top 3 rows



In [42]:
regComparisonDF.rdd.take(2)

[Row(rating=3.0, prediction=2.795339822769165),
 Row(rating=1.0, prediction=2.1862659454345703)]

In [43]:
print(type(regComparisonDF), type(regComparisonDF.rdd))

(<class 'pyspark.sql.dataframe.DataFrame'>, <class 'pyspark.rdd.RDD'>)


In [44]:
regComparisonrdd = regComparisonDF.rdd

In [45]:
regComparisonrdd.take(3)

[Row(rating=3.0, prediction=2.795339822769165),
 Row(rating=1.0, prediction=2.1862659454345703),
 Row(rating=1.0, prediction=1.1195515394210815)]

In [46]:
regComparisonrdd.map(lambda x: (x['rating'], x['prediction'])).take(3)

[(3.0, 2.795339822769165),
 (1.0, 2.1862659454345703),
 (1.0, 1.1195515394210815)]

In [47]:
regComparison = regComparisonrdd.map(lambda x: (x['rating'], x['prediction']))

In [48]:
metrics = RegressionMetrics(regComparison)

In [49]:
type(metrics)

pyspark.mllib.evaluation.RegressionMetrics

In [50]:
print(metrics.explainedVariance, 
      metrics.meanAbsoluteError, 
      metrics.meanSquaredError, 
      metrics.r2, 
      metrics.rootMeanSquaredError)

(1.5189450464766519, 0.9353079645051843, 1.50691832476769, 0.15837192319355942, 1.2275660164600883)


In [51]:
# COMMAND ----------
from pyspark.mllib.evaluation import RankingMetrics
from pyspark.sql.functions import col, expr

In [52]:
perUserActual = predictions\
  .where("rating > 2.5")\
  .groupBy("userId")\
  .agg(expr("collect_set(movieId) as movies"))

In [53]:
perUserActual.show(5)

+------+------------+
|userId|      movies|
+------+------------+
|    28|     [0, 12]|
|    26|[18, 36, 94]|
|    27|    [33, 83]|
|    12|    [66, 91]|
|    22|    [30, 80]|
+------+------------+
only showing top 5 rows



In [55]:
predictions.count()

168

In [54]:
perUserActual.count()

21

In [56]:
# COMMAND ----------
perUserPredictions = predictions\
  .orderBy(col("userId"), expr("prediction DESC"))\
  .groupBy("userId")\
  .agg(expr("collect_list(movieId) as movies"))

In [57]:
perUserPredictions.show(5)

+------+--------------------+
|userId|              movies|
+------+--------------------+
|    28|[98, 29, 94, 0, 4...|
|    26|[44, 40, 47, 85, ...|
|    27|[59, 95, 33, 25, ...|
|    12|[67, 66, 52, 60, ...|
|    22|[40, 80, 78, 19, ...|
+------+--------------------+
only showing top 5 rows



In [65]:
joinedDF = perUserActual.withColumnRenamed("movies","actualMovieIds")\
            .join(perUserPredictions.withColumnRenamed("movies","predicatedMovieIds"), ["userId"])

In [66]:
joinedDF.show(5)

+------+--------------+--------------------+
|userId|actualMovieIds|  predicatedMovieIds|
+------+--------------+--------------------+
|    28|       [0, 12]|[98, 29, 94, 45, ...|
|    26|  [18, 36, 94]|[47, 0, 36, 94, 4...|
|    27|      [33, 83]|[59, 90, 95, 93, ...|
|    12|      [66, 91]|[67, 60, 66, 91, ...|
|    22|      [30, 80]|[19, 40, 3, 80, 3...|
+------+--------------+--------------------+
only showing top 5 rows



In [72]:
joinedRDD = joinedDF.rdd.map(lambda row: (row[1], row[2][:2]))

In [73]:
joinedRDD.take(5)

[([0, 12], [45, 29]),
 ([18, 36, 94], [44, 47]),
 ([33, 83], [59, 25]),
 ([66, 91], [52, 67]),
 ([30, 80], [78, 19])]

In [74]:
# COMMAND ----------
perUserActualvPred = joinedDF.rdd.map(lambda row: (row[1], row[2][:15]))

In [75]:
ranks = RankingMetrics(perUserActualvPred)

In [76]:
# COMMAND ----------
ranks.meanAveragePrecision

0.38150037792894925

In [77]:
ranks.precisionAt(5)
# COMMAND ----------

0.37142857142857133